In [1]:
from mdagent.subagents import SubAgentSettings, SubAgentInitializer
from mdagent.utils import PathRegistry

pathregistry = PathRegistry.get_instance()
settings = SubAgentSettings(
    path_registry=pathregistry, subagents_model="gpt-3.5-turbo",temp=0.1
)
initializer = SubAgentInitializer(settings=settings)

In [2]:
## skill agent
example_fxn_code = '''
def get_pdb(query_string, PathRegistry):
    """
    Search RSCB's protein data bank using the given query string
    and return the path to pdb file
    """

    url = "https://search.rcsb.org/rcsbsearch/v2/query?json={search-request}"
    query = {
        "query": {
            "type": "terminal",
            "service": "full_text",
            "parameters": {"value": query_string},
        },
        "return_type": "entry",
    }
    r = requests.post(url, json=query)
    if r.status_code == 204:
        return "No Content Error: PDB ID not found for this substance."
    elif "result_set" in r.json() and len(r.json()["result_set"]) > 0:
        pdbid = r.json()["result_set"][0]["identifier"]
        url = f"https://files.rcsb.org/download/{pdbid}.cif"
        pdb = requests.get(url)
        filename = f"{pdbid}.cif"
        with open(filename, "w") as file:
            file.write(pdb.text)
        return filename
'''

skill_agent = initializer.create_skill_agent()

In [ ]:
result = skill_agent.add_new_tool(example_fxn_code)

In [4]:
len(skill_agent.get_skills())